# Required python packages

In [1]:
import pandas as pd
from datetime import datetime
import warnings

# Suppress any unimportant warnings

In [2]:
warnings.filterwarnings("ignore")

# Setting date format DD-MM-YYYY for "Order Date" and "Ship Date"

In [12]:
data = pd.read_csv("data_v1.csv")
dates_data = pd.read_csv('dates.csv')
col, row = dates_data.shape
for i in range(col):
    for j in range(row):
        if "/" in dates_data.iloc[i,j]:
            month, day, year = map(int, dates_data.iloc[i,j].split('/'))
            string = str(day)+"/"+str(month)+"/"+str(year)
            date_string = datetime.strptime(string, "%d/%m/%Y").strftime("%d-%m-%Y")
            dates_data.iloc[i,j] = date_string
        else:
            date_string = datetime.strptime(dates_data.iloc[i,j], "%d-%m-%Y").strftime("%d-%m-%Y")
            dates_data.iloc[i,j] = date_string

data['Order Date'] = dates_data['Order Date']
data['Ship Date'] = dates_data['Ship Date']

# The "Order Priority" columns has four different types of values as found below with their respective frequencies

In [15]:
data['Order Priority'].value_counts()

H    30
L    27
C    22
M    21
Name: Order Priority, dtype: int64

# Below code block will rename "C" as "Critical", "H" as "High", "M" as "Medium" and "L" as "Low"

In [16]:
for i in range(len(data['Order Priority'])):
    if data.iloc[i, 4] == "C":
        data.iloc[i, 4] = "Critical"
    elif data.iloc[i, 4] == "H":
        data.iloc[i, 4] = "High"
    elif data.iloc[i, 4] == "M":
        data.iloc[i, 4] = "Medium"
    else:
        data.iloc[i, 4] = "Low"

# Check for NAN
#### Below code shows that there are not any NULL values

In [17]:
data.isna().sum()

Region            0
Country           0
Item Type         0
Sales Channel     0
Order Priority    0
Order Date        0
Order ID          0
Ship Date         0
Units Sold        0
Unit Price        0
Unit Cost         0
Total Revenue     0
Total Cost        0
Total Profit      0
dtype: int64

# Check for duplicates
#### Below code shows that there are no duplicate entries

In [18]:
data.duplicated().sum()

0

# Sign validation
#### The fields Order ID, Units Sold, Unit Price, Unit Cost, Total Revenue can not be negative

In [19]:
def sign_validate(column_index):
    negative_value_count = 0
    invalid_message = "Invalid value found for column index " + str(column_index)
    valid_message = "Column with index " + str(column_index) + " is valid"
    for i in range(len(data['Order Priority'])):
        if data.iloc[i, column_index]<0:
            negative_value_count = negative_value_count + 1
    if negative_value_count > 0:
        return invalid_message
    else:
        return valid_message

In [21]:
sign_validate(6), sign_validate(8), sign_validate(9), sign_validate(10), sign_validate(11)

('Column with index 6 is valid',
 'Column with index 8 is valid',
 'Column with index 9 is valid',
 'Column with index 10 is valid',
 'Column with index 11 is valid')

#### Now all fields are valid. Further analysis is performed in Tableau

# Saving the corrected data

In [22]:
data.to_csv('data_v2.csv', index=False)